# Title
[]()

In [2]:

import sys
sys.path.append(r"/home/silvhua/custom_python")
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/')
sys.path.append(r'/home/silvhua/repositories/GHL-chat/src/app')
from silvhua import *
from chat_functions import *
from langchain.agents import Tool
from data_functions import parse_json_string

In [4]:
from app.ghl_requests import *
from dotenv import load_dotenv
load_dotenv(dotenv_path='../src/.env')

True

In [5]:
models_list = openai_models()

gpt-3.5-turbo-instruct-0914
gpt-3.5-turbo-instruct
gpt-3.5-turbo-0301
gpt-4-1106-preview
gpt-3.5-turbo-16k-0613
gpt-4-0613
gpt-3.5-turbo-16k
gpt-4
gpt-3.5-turbo
gpt-3.5-turbo-0613
gpt-3.5-turbo-1106
gpt-3.5-turbo-0125
gpt-4-vision-preview
gpt-4-0125-preview
gpt-4-turbo-preview


In [2]:
path = '/home/silvhua/repositories/GHL-chat/src/app/private'
contacts = load_json('contacts.json', path)
contactId = contacts['me_fb']

In [3]:
system_message_dict = dict()
conversation_dict = dict()
reply_dict = dict()
conversation_id = 1
question_id = 1
reply_dict[conversation_id] = dict()
inbound_dict = dict()

In [4]:
from app.ghl_requests import *
from dotenv import load_dotenv
load_dotenv(dotenv_path='../src/.env')
response = refresh_token()

Tokens retrieved from S3 for CoachMcloone.
Tokens saved to S3 CoachMcloone.


# Copied from 2024-02-04 notebook

In [ ]:
def create_test_system_message():
    system_message = f"""
You work for an online personal training business. 
Your role is to reach out to leads and book qualified leads for a call to see if they are a good fit 
for the business's online coaching program.
If the contact provides their telephone number, extract that phone number.

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage, if applicable. If a human is to be alerted, the value will be [ALERT HUMAN]
- "alert_human" (True or False): Whether or not to alert a human to review the response.
- "phone_number" (string or None): The phone number of the contact, if available.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    return system_message

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_test_system_message()
    conversation_dict[conversation_id] = create_chatbot(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )

    reply_dict[conversation_id][question_id] = chat_with_chatbot(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"This function does not do anything.",
    )
]
InboundMessage = 'My number is 7788917189'
conversation_id = 1
question_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, InboundMessage, tools)

# From 2024-02-10 notebook

In [ ]:
def create_test_system_message(
        business_name='CoachMcloone', 
        prompts_filepath='/home/silvhua/repositories/GHL-chat/src/app/private/prompts',
        examples_filepath='/home/silvhua/repositories/GHL-chat/src/app/private/data/chat_examples', 
        doc_filepath='/home/silvhua/repositories/GHL-chat/src/app/private/data/rag_docs'
        ):
    instructions_filename = f'{business_name}.md'
    examples_filename = f'{business_name}.txt'
    document_filename = f'{business_name}_doc.md'
    try:
        instructions = load_txt(instructions_filename, prompts_filepath)
        examples = load_txt(examples_filename, examples_filepath)
        document = load_txt(document_filename, doc_filepath)
    except Exception as error:
        if cloud == True:
            print(f'Error: {error}')
            print('Loading prompt files from s3...')
        s3 = boto3.client('s3')
        instructions = s3.get_object(
            Bucket='ownitfit-silvhua', Key=instructions_filename
            )['Body'].read().decode('utf-8')
        examples = s3.get_object(
            Bucket='ownitfit-silvhua', Key=examples_filename
            )['Body'].read().decode('utf-8')
        document = s3.get_object(
            Bucket='ownitfit-silvhua', Key=document_filename
            )['Body'].read().decode('utf-8')
    # print(f'**Instructions component of system message**: \n{instructions}\n')

    system_message = f"""{instructions}

## Other Messages

Only repond to inbound messages that can be answered by the message templates or provided 
documenation. Otherwise, return "[ALERT HUMAN]". 
If the message indicates the contact has an eating disorder, suicidal ideation, or other serious mental health 
conditions, return "[ALERT HUMAN]". 
The "[ALERT HUMAN]" message will trigger a human staff member to review the messages to write a response. 
It is better to err on the side of caution and flag a staff rather than give a wrong response.
    
# Stage 1

Determine if you should generate a response to the inbound message. If so, generate the response and proceed 
to Stage 2. Otherwise, return "[ALERT HUMAN]".

Return your response on a JSON format with the following keys:
- "response" (string): The response to the InboundMessage, if applicable. If a human is to be alerted, the value will be [ALERT HUMAN]
- "alert_human" (True or False): Whether or not to alert a human to review the response.
- "phone_number" (string or None): The phone number of the contact, if available.

## Examples

Below are example conversations with leads. Each lead as a unique contact ID.
An InboundMessage is from the lead. An OutboundMessage is from you.

{examples}

## Relevant documentation

{document}

# Stage 2

Review your response from stage 1. 
Revise your response if needed to make sure you followed the instructions.
Revise your response if needed to avoid asking questions that have already been answered in previous messages.
Make sure that if the question cannot be answered through the message templates or documentation, 
you return "[ALERT HUMAN]".

# Stage 3

Review your response from stage 2 to revise as needed to make it concise.
    """

    prompt = """
    Write the next OutboundMessage based on the following InboundMessage, 
    which is delimited by triple backticks: ```{InboundMessage}```
    """
    system_message = f'{system_message}{prompt}'
    ###
    # print(f'\n**System_message**: {system_message}\n\n')
    return system_message

def test_chatbot(contactId, InboundMessage, tools):
    system_message_dict[conversation_id] = create_test_system_message()
    conversation_dict[conversation_id] = create_chatbot(
        contactId, system_message_dict[conversation_id], tools=tools,
        # model='gpt-4-32k'
        )

    reply_dict[conversation_id][question_id] = chat_with_chatbot(
        InboundMessage, conversation_dict[conversation_id]
    )
    chatbot_response = parse_json_string(reply_dict[conversation_id][question_id]["output"])

    return chatbot_response


tools = [
    Tool(
        name=f"placeholder_function",
        func=placeholder_function,
        description=f"This function does not do anything.",
    )
]
inbound_id = 1
inbound_dict[inbound_id] = 'hi'
conversation_id = 1
reply_dict[conversation_id] = dict()
chatbot_response = test_chatbot(contactId, inbound_dict[inbound_id], tools)
chatbot_response

# *End of Page*